# Evaluation with new Inference with limit of input seq length

In [1]:
import torch
import numpy as np
import data_visualization.Visualization as visualization

# Check if GPU is available, set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [2]:
# Define Data parameters

sos_token = np.full((1, 24), 1)
pad_token = np.full((1, 24), 2)
pad_token = torch.tensor(pad_token, device=device)

dataset_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/evaluation/specific_test_midis/split_into_mid"
snapshot_intervall = 0.05

batch_size = 1
seq_length = 2048
stride = seq_length

test_size=0.1

In [3]:
from transformer_decoder_training.dataprep_transformer.prepare_dataloader_complete import prepare_dataset_as_single_loader

# Load Data

#train_loader, val_loader, test_loader = prepare_dataset_as_dataloaders(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, test_size, sos_token)
train_loader = prepare_dataset_as_single_loader(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, sos_token, shuffle=False)

Processed dataset (2/2): 100%|██████████| 2/2 [00:00<00:00,  7.00it/s]

Processed 2 of 2 files


In [4]:
from transformer_decoder_training.transformer_inference_eval import inference_and_visualize_1

# Get sequence
sequence = next(iter(train_loader))
print(sequence.shape)

context_seq, continuing_seq, original_seq = inference_and_visualize_1.prepare_sequence(sequence, 513)

torch.Size([1, 2049, 24])


# 6.1

Model with BCELossWithlogitsloss

Load model

In [5]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.1.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

## Inference

In [6]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1"

In [7]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.17, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")


Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1/limited_input_seq_513.mid


In [8]:
# inference with threshold and limited input seq
# limit = 1024 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.17, pad_token, 1024, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_1024.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1/limited_input_seq_1024.mid


# 6.2

Model with basic CrossentropyLoss

In [9]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.2.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

In [10]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2"

In [11]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.999, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2/limited_input_seq_513.mid


In [12]:
# inference with threshold and limited input seq
# limit = 1024 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.999, pad_token, 1024, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_1024.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2/limited_input_seq_1024.mid


# 6.3

Model with melody + Harmony as input and Harmony only as output

In [13]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads, num_output_dim=12).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.3.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=12, bias=True)
)

In [14]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.3"

In [15]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference_melody_only(model, context_seq, continuing_seq, 0.17, pad_token, 513, device)

print("last generated token: ", tokens_with_truth[-1].shape)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
last generated token:  torch.Size([1, 24])
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.3/limited_input_seq_513.mid


# 6.4

ist modell mit MSE loss

In [16]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads, num_output_dim=12).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.4.pth"))
model.eval()

midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.4"

tokens_with_truth, generated_logits = inference_5.inference_melody_only(model, context_seq, continuing_seq, 0.7, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.4/limited_input_seq_513.mid


In [17]:
visualization.show_tensors_min_max_values(generated_logits)

Min value: -0.02013530768454075 Max value: 0.9533243179321289 Sigmoid Min: 0.4949663281440735 Sigmoid Max: 0.7217832207679749
Min value: -0.023811155930161476 Max value: 0.9588487148284912 Sigmoid Min: 0.49404746294021606 Sigmoid Max: 0.7228912711143494
Min value: -0.02424311451613903 Max value: 0.9602009057998657 Sigmoid Min: 0.49393948912620544 Sigmoid Max: 0.723162055015564
Min value: -0.02357570268213749 Max value: 0.959199070930481 Sigmoid Min: 0.49410638213157654 Sigmoid Max: 0.72296142578125
Min value: -0.023338811472058296 Max value: 0.9581949710845947 Sigmoid Min: 0.4941655099391937 Sigmoid Max: 0.7227602005004883
Min value: -0.02399589493870735 Max value: 0.9578123092651367 Sigmoid Min: 0.49400126934051514 Sigmoid Max: 0.7226835489273071
Min value: -0.024900389835238457 Max value: 0.9583256244659424 Sigmoid Min: 0.49377521872520447 Sigmoid Max: 0.7227864265441895
Min value: -0.04013815522193909 Max value: 0.9739795923233032 Sigmoid Min: 0.48996680974960327 Sigmoid Max: 0.7259

In [23]:
print(generated_logits[0])
print(generated_logits[1])

tensor([[-0.0035,  0.0267,  0.0013,  0.9470,  0.0113,  0.0680,  0.0462,  0.0013,
          0.0302,  0.0127,  0.0358, -0.0189]], device='cuda:0')
tensor([[-0.0040,  0.0267,  0.0022,  0.9448,  0.0089,  0.0655,  0.0454,  0.0054,
          0.0258,  0.0063,  0.0355, -0.0241]], device='cuda:0')


# MAE Loss

In [19]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads, num_output_dim=12).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_mae_loss.pth"))
model.eval()

midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/mae_loss"

tokens_with_truth, generated_logits = inference_5.inference_melody_only(model, context_seq, continuing_seq, 0.7, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/mae_loss/limited_input_seq_513.mid


In [20]:
visualization.show_tensors_min_max_values(generated_logits)

Min value: -0.007641451433300972 Max value: 0.9514622688293457 Sigmoid Min: 0.49808961153030396 Sigmoid Max: 0.7214092016220093
Min value: -0.007651498541235924 Max value: 0.9517875909805298 Sigmoid Min: 0.4980871379375458 Sigmoid Max: 0.7214745283126831
Min value: -0.007634250447154045 Max value: 0.9519985914230347 Sigmoid Min: 0.4980914294719696 Sigmoid Max: 0.7215169072151184
Min value: -0.007604112848639488 Max value: 0.9521654844284058 Sigmoid Min: 0.4980989396572113 Sigmoid Max: 0.721550464630127
Min value: -0.007579430937767029 Max value: 0.952446699142456 Sigmoid Min: 0.49810510873794556 Sigmoid Max: 0.721606969833374
Min value: -0.0075870659202337265 Max value: 0.952921450138092 Sigmoid Min: 0.49810320138931274 Sigmoid Max: 0.7217023372650146
Min value: -0.007630564272403717 Max value: 0.9534626603126526 Sigmoid Min: 0.49809232354164124 Sigmoid Max: 0.7218109965324402
Min value: -0.00794648751616478 Max value: 0.9537590146064758 Sigmoid Min: 0.49801337718963623 Sigmoid Max: 0.

# Huber loss

In [21]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads, num_output_dim=12).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_huber_loss.pth"))
model.eval()

midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/huber_loss"

tokens_with_truth, generated_logits = inference_5.inference_melody_only(model, context_seq, continuing_seq, 0.7, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/huber_loss/limited_input_seq_513.mid


In [22]:
visualization.show_tensors_min_max_values(generated_logits)

Min value: -0.018892135471105576 Max value: 0.9470427632331848 Sigmoid Min: 0.49527713656425476 Sigmoid Max: 0.7205200791358948
Min value: -0.02410826086997986 Max value: 0.9447628259658813 Sigmoid Min: 0.4939732253551483 Sigmoid Max: 0.7200607061386108
Min value: -0.025507893413305283 Max value: 0.9416208863258362 Sigmoid Min: 0.49362337589263916 Sigmoid Max: 0.7194269299507141
Min value: -0.02513522282242775 Max value: 0.9403111338615417 Sigmoid Min: 0.4937165081501007 Sigmoid Max: 0.7191625237464905
Min value: -0.024930953979492188 Max value: 0.9378988742828369 Sigmoid Min: 0.4937676191329956 Sigmoid Max: 0.7186750173568726
Min value: -0.025561824440956116 Max value: 0.9372805953025818 Sigmoid Min: 0.4936099052429199 Sigmoid Max: 0.7185500264167786
Min value: -0.02751535177230835 Max value: 0.9437719583511353 Sigmoid Min: 0.49312159419059753 Sigmoid Max: 0.7198609709739685
Min value: -0.021355479955673218 Max value: 0.9775362610816956 Sigmoid Min: 0.4946613013744354 Sigmoid Max: 0.7